## Notebook Setup 
The following cell will install Drake, checkout the underactuated repository, and set up the path (only if necessary).
- On Google's Colaboratory, this **will take approximately two minutes** on the first time it runs (to provision the machine), but should only need to reinstall once every 12 hours.  Colab will ask you to "Reset all runtimes"; say no to save yourself the reinstall.
- On Binder, the machines should already be provisioned by the time you can run this; it should return (almost) instantly.

More details are available [here](http://underactuated.mit.edu/underactuated.html?chapter=drake).

In [ ]:
try:
    import pydrake
    import underactuated
except ImportError:
    !curl -s https://raw.githubusercontent.com/RussTedrake/underactuated/master/scripts/setup/jupyter_setup.py > jupyter_setup.py
    from jupyter_setup import setup_underactuated
    setup_underactuated()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pydrake.all import MathematicalProgram, Solve, Variables, Polynomial

# System dynamics

In [ ]:
# Scalar dynamics.
f = lambda x, u : x - 4 * x ** 3 + u

# Quadratic running cost.
l = lambda x, u : x ** 2 + u ** 2

# Input limits.
U = [-1, 1]

# State limits (region of state space where we approximate the value function).
X = [-1, 1]

In [ ]:
# Plot dynamics with zero input.
n_breaks = 101
x_breaks = np.linspace(*X, n_breaks)
plt.plot(x_breaks, f(x_breaks, np.zeros(n_breaks)))
plt.xlabel(r'$x$')
plt.ylabel(r'$f(x, u=0)$')
plt.grid(True)

# Auxiliary function to integrate polynomials over an interval

In [ ]:
from sympy import Symbol, integrate

# Function that integrates a (multivariate) polynomial p(x) over the interval [x_min, x_max].
# Needed in the objective functions of the SOS program .
def polyint(p, x_min, x_max):
    
    # integration variables
    nx = len(x_min)
    assert(len(x_max) == nx)
    x = [Symbol(f'x({i})') for i in range(nx)]

    # compute integral one monomial per time
    integral = 0
    for m, c in p.monomial_to_coefficient_map().items():

        # integrand for the current monomial
        m_integrand = 1
        for i, xi in enumerate(p.indeterminates()):
            m_integrand *= x[i] ** m.degree(xi)

        # numeric value of the integral of the monomial
        m_integral = m_integrand
        for i, x_i in enumerate(x):
            m_integral = integrate(m_integral, (x_i, x_min[i], x_max[i]))

        # add monomial integral to the overall polynomial integral
        integral += c * float(m_integral)

    return integral

# Lower bound on the value function

In [ ]:
# Given the degree for the approximate value function and the polynomials
# in the S procedure, solves the SOS and returns the approximate value function
# (together with the objective of the SOS program).
def approximate_dp(deg):
    
    # Set up SOS program.
    prog = MathematicalProgram()
    x = prog.NewIndeterminates(1, 'x')[0]
    u = prog.NewIndeterminates(1, 'u')[0]
    v = prog.NewFreePolynomial(Variables([x]), deg)

    # Maximize volume beneath the value function.
    v_int = polyint(v, *[[Xi] for Xi in X])
    prog.AddLinearCost(- v_int)
    
    # S-procedure for the input limits.
    xu = Variables([x, u])
    lamx = prog.NewSosPolynomial(xu, deg)[0]
    S_procedure = lamx * Polynomial((x - X[0]) * (X[1] - x))
    
    # S-procedure for the input limits.
    lamu = prog.NewSosPolynomial(xu, deg)[0]
    S_procedure += lamu * Polynomial((u - U[0]) * (U[1] - u))
    
    # Enforce Bellman inequality.
    v_dot = v.Differentiate(x) * Polynomial(f(x, u))
    prog.AddSosConstraint(v_dot + Polynomial(l(x, u)) - S_procedure)

    # v(0) = 0.
    prog.AddLinearConstraint(v.EvaluatePartial({x: 0}).ToExpression() == 0)

    # Solve and retrieve result.
    result = Solve(prog)
    assert result.is_success()

    # retrieve value function
    v_opt_expr = result.GetSolution(v.ToExpression())
    v_opt = lambda x_eval: v_opt_expr.Evaluate({x: x_eval})
    cost = - result.get_optimal_cost()
    
    return v_opt, cost

In [ ]:
# Solve for increasing degree.
degrees = np.arange(1, 9) * 2
v = {deg: approximate_dp(deg) for deg in degrees}

In [ ]:
# Plot solution.
plt.figure()
for deg in degrees:
    label = f'Deg. {deg}, obj. {round(v[deg][1], 3)}'
    v_plot = [v[deg][0](xi) for xi in x_breaks]
    plt.plot(x_breaks, v_plot, label=label)
    plt.ylabel(r'$J_{\mathrm{lb}}$')
    plt.title('Value-function lower bound')
    plt.legend()